In [ ]:
# importamos las librerías a utilizar
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import metrics
from sklearn import preprocessing
import pickle
import json
import numpy as np

In [ ]:
# Definimos las columnas que no vamos a tomar en cuenta
cols_remove = ['mantenimiento', 'fecha']
# Definimos las columnas que van a ser los datos de entrada (X)
cols_X = ['vel_motor', 'voltaje', 'mensaje_volt', 'horas_comp_trabajadas',
 'temp_planta', 'Radiation_mrh', 'problemas_rechazo'] #'Lamps', 'Lead Curtains',
# Definimos las variables que vamos a predecir (y)
cols_y = ['rep_filtro_aire', 'rep_tubo', 'rep_tarjeta_detector', 'rep_IOB',
 'rep_otras_tarj', 'rep_motor', 'rep_software', 'rep_PILZ', 'rep_shutter',
 'rep_cables']

In [ ]:
# importamos el archivo de un equipo #Serie E162533
init_df = pd.read_excel("equipo-E162533_v2.xlsx")
init_df.head()

In [ ]:
# generamos los dataframes para 'X' e 'y'
df_X = init_df.copy()[cols_X]
df_y = init_df.copy()[cols_y]

In [ ]:
#normalizamos los valores de 'X' que vamos a utilizar
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(df_X)
std_df_X = pd.DataFrame(mm_scaler.transform(df_X), columns = cols_X)
std_df_X.head()

In [ ]:
# normalizamos los valores de 'y'
mm_scaler = preprocessing.MinMaxScaler()
X_train_minmax = mm_scaler.fit_transform(df_y)
std_df_y = pd.DataFrame(mm_scaler.transform(df_y), columns = cols_y)
std_df_y.head()

- https://arxiv.org/pdf/1407.7502.pdf
- https://towardsdatascience.com/understanding-random-forest-58381e0602d2
- https://www.cienciadedatos.net/documentos/py08_random_forest_python.html

In [ ]:
def calcula_modelo_rf(df_X, df_Y, test_size=0.2, n_estimators = 100, 
                      criterion='gini',  min_samples_leaf=1, 
                      min_samples_split=2, n_jobs=1, logistic_model=False):
    '''
    '''
    # Creamos un objeto donde vamos a ir guardando los resultados
    results = {
        'y': df_Y.name
    }
    
    # dividimos en sets de entrenamiento y de prueba
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=test_size)
    
    if (logistic_model):
        # creamos un Regresor Logístico
        clf = RandomForestRegressor(n_estimators=n_estimators,criterion=criterion,
            max_depth=None,max_samples=0.7,oob_score=False,
            n_jobs=-1,random_state=6)
    else:
        # creamos un Clasificador Gaussiano
        clf = RandomForestClassifier(n_estimators=n_estimators)
    
    # Entrenamos el modelo utilizando los sets de entrenamiento
    clf.fit(X_train,y_train)
    
    # Si save_model=True entonces guardamos el modelo como pickle
    if(logistic_model):
        filename = f'model_{df_Y.name}.sav'
        pickle.dump(clf, open(filename, 'wb'))
        results['model'] = filename
    
    # Utilizamos el modelo para crear una predicción
    y_pred=clf.predict(X_test)
    
    if(logistic_model):
        # obtenemos el error medio cuadrático
        results['mse'] = metrics.mean_squared_error(y_true=y_test,y_pred=y_pred)
    else:
        # obtenemos la precisión  y la guardamos en los resultados
        results['accuracy'] = metrics.accuracy_score(y_test, y_pred)
    
    # creamos el clasificador para obtener las variables relevantes para esta 'y'
    RandomForestClassifier(bootstrap=True, class_weight=None, criterion=criterion,
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, 
            min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=n_jobs,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
    feature_imp = pd.Series(clf.feature_importances_,index=df_X.columns).sort_values(ascending=False)
    
    # Guardamos las primeras 5 columnas, que son las más representativas
    results['columnas'] = feature_imp.head().to_dict()
    
    # Regresamos el resultado del modelo
    return results

In [ ]:
# Para cada 'y' vamos a obtener cuales son las 'X' más representativas
results = []

for y in cols_y:
    df_aux_Y = std_df_y.copy()[y]
    print(df_aux_Y.name)
    results.append(calcula_modelo_rf(std_df_X,df_aux_Y,0.2,100,'gini',1,2,1,False))

# Imprimimos los resultados
#results

In [ ]:
# ya con las variables 'X' más importantes para cada 'y' ahora si entrenamos un modelo
# que de una sola predicción y que sea interpretable
final_results = []

for obj in results:
    # obtenemos los dataframes auxiliares con los datos de los resultados anteriores
    df_aux_Y = std_df_y.copy()[obj['y']]
    df_aux_X = std_df_X.copy()[obj['columnas'].keys()]
    print(df_aux_Y.name)
    print(df_aux_X.columns)
    # Guardamos los resultados para revisar el MSE
    final_results.append(calcula_modelo_rf(df_aux_X,df_aux_Y,0.2,100,'squared_error',1,2,1,True))
    
# Imprimimos los resultados
#final_results

In [ ]:
# guardamos los resultados en un JSON
with open('outputfile.json', 'w', encoding='utf-8') as fout:
    json.dump(final_results, fout)

In [ ]:
# leemos los resultados del JSON
f = open('outputfile.json')
data = json.load(f)
#data

### Prueba de los modelos

In [ ]:
# dividimos en sets de entrenamiento y de prueba
X_train, X_test, y_train, y_test = train_test_split(std_df_X, std_df_y, test_size=0.2)

# Para cargar el modelo guardado:
for y in data:
    
    # Obtenemos los dataframes con las columnas que nos interesan
    df_test_X = X_test.copy()[y['columnas'].keys()]
    df_test_y = y_test.copy()[y['y']]
    
    # cargamos el modelo guardado
    filename = y['model']
    model = pickle.load(open(filename, 'rb'))
    
    # checamos el orden de los features para ponerlos en el mismo orden
    cols_when_model_builds = model.feature_names_in_
    #print(f'outputs: {model.n_outputs_}')
    
    # Creamos el dataframe con los features en el orden que deben de ir
    df_test_X = pd.DataFrame(df_test_X,columns=cols_when_model_builds)
    
    # para hacer la predicción pasamos el dataframe con los valores de prueba recien generados
    y_pred = model.predict(df_test_X)
    
    # la predicción del RandomForest es el promedio de los outputs
    y_pred_int = np.rint(np.average(y_pred, axis=1))
    
    # comparamos el error cuadrado medio
    mse = metrics.mean_squared_error(df_test_y, y_pred_int)
    print(f'Model: {filename} - MSE: {mse}') #\n Pred_int: \n {y_pred_int} \n vs real:\n {df_test_y} - MSE: {mse}')# \n predicción:\n {y_pred}